<a href="https://colab.research.google.com/github/Soham109/Big-5-personality-trait-detector/blob/main/Model_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers scikit-learn xgboost datasets catboost umap-learn
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import StackingRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import numpy as np
import pandas as pd
from datasets import load_dataset
import umap.umap_ as umap


In [ ]:

big5 = load_dataset('Fatima0923/Automated-Personality-Prediction')

sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def extract_embeddings(dataset):
    return np.array([sbert_model.encode(text) for text in dataset['text']])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/845 [00:00<?, ?B/s]

(…)sonality Datasets - Reddit/train_set.csv:   0%|          | 0.00/4.63M [00:00<?, ?B/s]

(…)ersonality Datasets - Reddit/val_set.csv:   0%|          | 0.00/732k [00:00<?, ?B/s]

(…)rsonality Datasets - Reddit/eval_set.csv:   0%|          | 0.00/663k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
train_embeddings = extract_embeddings(big5['train'])
validation_embeddings = extract_embeddings(big5['validation'])
test_embeddings = extract_embeddings(big5['test'])


In [ ]:

umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=50, random_state=42)
train_embeddings_reduced = umap_model.fit_transform(train_embeddings)
validation_embeddings_reduced = umap_model.transform(validation_embeddings)
test_embeddings_reduced = umap_model.transform(test_embeddings)

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [ ]:
scaler = StandardScaler()
train_embeddings_standardized = scaler.fit_transform(train_embeddings_reduced)
validation_embeddings_standardized = scaler.transform(validation_embeddings_reduced)
test_embeddings_standardized = scaler.transform(test_embeddings_reduced)


In [ ]:
tfidf = TfidfVectorizer(max_features=1000)
train_texts = big5['train']['text']
validation_texts = big5['validation']['text']
test_texts = big5['test']['text']


In [ ]:
train_tfidf = tfidf.fit_transform(train_texts).toarray()
validation_tfidf = tfidf.transform(validation_texts).toarray()
test_tfidf = tfidf.transform(test_texts).toarray()


In [ ]:
train_combined = np.hstack((train_embeddings_standardized, train_tfidf))
validation_combined = np.hstack((validation_embeddings_standardized, validation_tfidf))
test_combined = np.hstack((test_embeddings_standardized, test_tfidf))


In [ ]:
y_train = big5['train'].to_pandas()[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']]
y_validation = big5['validation'].to_pandas()[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']]
y_test = big5['test'].to_pandas()[['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism']]


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

param_grid = {
    'estimator__learning_rate': [0.1],
    'estimator__n_estimators': [200],
    'estimator__max_depth': [9],
    'estimator__subsample': [1.0],
    'estimator__colsample_bytree': [0.8],
    'estimator__reg_lambda': [1],
    'estimator__reg_alpha': [1.5]
}


xgb_base = XGBRegressor(objective='reg:squarederror', n_jobs=-1)

xgb_model = RandomizedSearchCV(
    MultiOutputRegressor(xgb_base),
    param_distributions=param_grid,
    n_iter=1,
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42,
    verbose=3
)


xgb_model.fit(train_combined, y_train)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.1, estimator__max_depth=9, estimator__n_estimators=200, estimator__reg_alpha=1.5, estimator__reg_lambda=1, estimator__subsample=1.0;, score=-768.728 total time= 3.8min
[CV 2/3] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.1, estimator__max_depth=9, estimator__n_estimators=200, estimator__reg_alpha=1.5, estimator__reg_lambda=1, estimator__subsample=1.0;, score=-757.334 total time= 3.8min
[CV 3/3] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.1, estimator__max_depth=9, estimator__n_estimators=200, estimator__reg_alpha=1.5, estimator__reg_lambda=1, estimator__subsample=1.0;, score=-771.608 total time= 3.6min


RandomizedSearchCV(cv=3,
                   estimator=MultiOutputRegressor(estimator=XGBRegressor(base_score=None,
                                                                         booster=None,
                                                                         callbacks=None,
                                                                         colsample_bylevel=None,
                                                                         colsample_bynode=None,
                                                                         colsample_bytree=None,
                                                                         device=None,
                                                                         early_stopping_rounds=None,
                                                                         enable_categorical=False,
                                                                         eval_metric=None,
                                                                         feature_types=None,
                                                                         gamma=None,
                                                                         grow_policy=None,
                                                                         importance_type=None,
                                                                         interaction_...
                                                                         num_parallel_tree=None,
                                                                         random_state=None, ...)),
                   n_iter=1,
                   param_distributions={'estimator__colsample_bytree': [0.8],
                                        'estimator__learning_rate': [0.1],
                                        'estimator__max_depth': [9],
                                        'estimator__n_estimators': [200],
                                        'estimator__reg_alpha': [1.5],
                                        'estimator__reg_lambda': [1],
                                        'estimator__subsample': [1.0]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=3)

In [ ]:
y_val_pred_xgb = xgb_model.predict(validation_combined)
mse_xgb = mean_squared_error(y_validation, y_val_pred_xgb, multioutput='raw_values')
average_mse_xgb = mean_squared_error(y_validation, y_val_pred_xgb)

print("XGBoost MSE for each trait:", mse_xgb)
print("XGBoost Average MSE:", average_mse_xgb)
y_test_pred_xgb = xgb_model.predict(test_combined)
test_mse_xgb = mean_squared_error(y_test, y_test_pred_xgb, multioutput='raw_values')
average_test_mse_xgb = mean_squared_error(y_test, y_test_pred_xgb)
print("XGBoost Test MSE for each trait:", test_mse_xgb)
print("XGBoost Average Test MSE:", average_test_mse_xgb)

XGBoost MSE for each trait: [628.43075386 410.82522095 504.65662896 619.6749557  527.9309906 ]
XGBoost Average MSE: 538.3037100157909
XGBoost Test MSE for each trait: [620.96371437 392.0308595  469.43561542 622.88576695 516.17074778]
XGBoost Average Test MSE: 524.2973408046048


In [ ]:

lgbm_model = MultiOutputRegressor(LGBMRegressor())
catboost_model = MultiOutputRegressor(CatBoostRegressor(verbose=0))

In [ ]:
lgbm_model.fit(train_combined, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.279830 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 91912
[LightGBM] [Info] Number of data points in the train set: 16047, number of used features: 1049
[LightGBM] [Info] Start training from score 39.036767
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.245348 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 91912
[LightGBM] [Info] Number of data points in the train set: 16047, number of used features: 1049
[LightGBM] [Info] Start training from score 67.662024
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.288840 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 91912
[LightGBM] [Info] Number of data points in the train set: 16047, number of used features: 1049
[LightGBM] [Info

MultiOutputRegressor(estimator=LGBMRegressor())

In [ ]:
y_val_pred_lgbm = lgbm_model.predict(validation_combined)
mse_lgbm = mean_squared_error(y_validation, y_val_pred_lgbm, multioutput='raw_values')
average_mse_lgbm = mean_squared_error(y_validation, y_val_pred_lgbm)

print("LightGBM MSE for each trait:", mse_lgbm)
print("LightGBM Average MSE:", average_mse_lgbm)

LightGBM MSE for each trait: [704.66218803 453.23213992 591.47114966 706.83154431 602.63767166]
LightGBM Average MSE: 611.7669387162198


In [ ]:
catboost_model.fit(train_combined, y_train)

Learning rate set to 0.063479
0:	learn: 29.3934003	total: 436ms	remaining: 7m 15s
3:	learn: 29.3186022	total: 1.38s	remaining: 5m 42s
6:	learn: 29.2456138	total: 2.25s	remaining: 5m 20s
9:	learn: 29.1898260	total: 3.18s	remaining: 5m 14s
12:	learn: 29.1430047	total: 4.09s	remaining: 5m 10s
15:	learn: 29.0806635	total: 5.05s	remaining: 5m 10s
18:	learn: 29.0444287	total: 6.01s	remaining: 5m 10s
21:	learn: 29.0115125	total: 6.88s	remaining: 5m 5s
24:	learn: 28.9827061	total: 7.87s	remaining: 5m 7s
27:	learn: 28.9436807	total: 9.18s	remaining: 5m 18s
30:	learn: 28.9150828	total: 10.5s	remaining: 5m 27s
33:	learn: 28.8886065	total: 11.9s	remaining: 5m 39s
36:	learn: 28.8529545	total: 13s	remaining: 5m 38s
39:	learn: 28.8246988	total: 13.9s	remaining: 5m 34s
42:	learn: 28.8019689	total: 14.8s	remaining: 5m 28s
45:	learn: 28.7775533	total: 15.7s	remaining: 5m 25s
48:	learn: 28.7559916	total: 16.6s	remaining: 5m 21s
51:	learn: 28.7365184	total: 17.6s	remaining: 5m 20s
54:	learn: 28.7128104	to

MultiOutputRegressor(estimator=<catboost.core.CatBoostRegressor object at 0x7b87ee04f460>)

In [ ]:
y_val_pred_catboost = catboost_model.predict(validation_combined)
mse_catboost = mean_squared_error(y_validation, y_val_pred_catboost, multioutput='raw_values')
average_mse_catboost = mean_squared_error(y_validation, y_val_pred_catboost)

print("CatBoost MSE for each trait:", mse_catboost)
print("CatBoost Average MSE:", average_mse_catboost)

# Predict and evaluate on test set
y_test_pred_catboost = catboost_model.predict(test_combined)
test_mse_catboost = mean_squared_error(y_test, y_test_pred_catboost, multioutput='raw_values')
average_test_mse_catboost = mean_squared_error(y_test, y_test_pred_catboost)
print("CatBoost Test MSE for each trait:", test_mse_catboost)
print("CatBoost Average Test MSE:", average_test_mse_catboost)

CatBoost MSE for each trait: [665.92042211 430.78415055 551.14753053 667.92860887 559.60741501]
CatBoost Average MSE: 575.0776254151009
CatBoost Test MSE for each trait: [654.40933719 416.29524533 525.14894643 677.97004522 545.93112681]
CatBoost Average Test MSE: 563.9509401961388
